# Import

## Library

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from PIL import Image
from keras.layers import Dropout
from tensorflow.keras import regularizers
import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.datasets import mnist

print("TensorFlow version:", tf.__version__)



TensorFlow version: 2.12.0


## Dataset

In [4]:
# Load and prepare the MNIST dataset
mnist = tf.keras.datasets.mnist
# Split dataset in data of Train and Data od Test
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_val, y_train, y_val  = train_test_split( x_train, y_train, test_size=0.166, random_state=4)
x_train,x_val, x_test = x_train / 255.0, x_val/ 255.0, x_test / 255.0

In [5]:
print(f'x_train.shape : {x_train.shape}')
print(f'y_train.shape : {y_train.shape}\n')

print(f'x_val.shape : {x_val.shape}')
print(f'y_val.shape : {y_val.shape}\n')

print(f'x_test.shape : {x_test.shape}')
print(f'y_test.shape : {y_test.shape}\n')

x_train.shape : (50040, 28, 28)
y_train.shape : (50040,)

x_val.shape : (9960, 28, 28)
y_val.shape : (9960,)

x_test.shape : (10000, 28, 28)
y_test.shape : (10000,)



![](MNIST.png)

# Keras tuners

In [9]:
# Para eliminar los modelos que hayan quedado guardados en memoria
tf.keras.backend.clear_session() 

In [11]:
# Definir la función de modelo para Keras Tuner
# hp: es el objeto hp que se utiliza para ajustar los hiperparámetros del modelo.
def build_model(hp):
    
    #se crea un modelo secuencial utilizando la biblioteca Keras.
    model = keras.Sequential()
    
    tf.keras.layers.Conv2D(75, (3,3), activation = "relu", input_shape= (28, 28, 1)),
    tf.keras.layers.MaxPool2D((2,2)),
    
    # Esta capa transforma los datos de entrada en un vector unidimensional.
    model.add(layers.Flatten(input_shape=(28, 28)))
    
    #se itera sobre un rango determinado por el valor de hiperparámetro num_layers. 
    #Este hiperparámetro define el número de capas ocultas en el modelo.
    for i in range(hp.Int('num_layers', min_value=1, max_value=5)):
        
        # se agregan capas densas (fully connected) al modelo. 
        # El número de unidades en cada capa se ajusta utilizando el 
        # hiperparámetro units con un rango específico definido por min_value, max_value y step.
        model.add( layers.Dense( units      = hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),
                                 activation = 'relu') )

    model.add(layers.Dense(10, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])


    return model

In [12]:
# Configurar el optimizador de búsqueda aleatoria
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=3,
    factor=3,
    project_name='mnist_tuner',
)



In [13]:
# Realizar la búsqueda de hiperparámetros
tuner.search(x_train, y_train,
             epochs=5,
             validation_data=(x_val, y_val))

# Obtener la mejor combinación de hiperparámetros
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 6 Complete [00h 00m 38s]
val_accuracy: 0.9762048125267029

Best val_accuracy So Far: 0.9763052463531494
Total elapsed time: 00h 03m 54s
INFO:tensorflow:Oracle triggered exit


In [14]:
# Construir el modelo con la mejor combinación de hiperparámetros
best_model = tuner.hypermodel.build(best_hps)

In [15]:
# Entrenar el modelo con la mejor combinación de hiperparámetros
best_model.fit(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

Epoch 1/5
1564/1564 [==============================] - 15s 9ms/step - loss: 0.2026 - accuracy: 0.9375 - val_loss: 0.1449 - val_accuracy: 0.9564
Epoch 2/5
1564/1564 [==============================] - 14s 9ms/step - loss: 0.0856 - accuracy: 0.9732 - val_loss: 0.1003 - val_accuracy: 0.9693
Epoch 3/5
1564/1564 [==============================] - 15s 10ms/step - loss: 0.0588 - accuracy: 0.9815 - val_loss: 0.0822 - val_accuracy: 0.9756
Epoch 4/5
1564/1564 [==============================] - 13s 8ms/step - loss: 0.0417 - accuracy: 0.9864 - val_loss: 0.0878 - val_accuracy: 0.9769
Epoch 5/5
1564/1564 [==============================] - 14s 9ms/step - loss: 0.0364 - accuracy: 0.9881 - val_loss: 0.0929 - val_accuracy: 0.9742


In [16]:
best_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 448)               351680    
                                                                 
 dense_3 (Dense)             (None, 256)               114944    
                                                                 
 dense_4 (Dense)             (None, 10)                2570      
                                                                 
Total params: 469,194
Trainable params: 469,194
Non-trainable params: 0
_________________________________________________________________
